In [1]:
#Fire up graphlab create
import graphlab
import json
import pandas as pd
import os
import numpy as np
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import string
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix
import random
pd.set_option('display.max_columns', 500)
from sklearn.metrics import pairwise_distances
from sklearn.metrics import confusion_matrix
from collections import defaultdict

This non-commercial license of GraphLab Create for academic use is assigned to sisterjiang520@gmail.com and will expire on October 24, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1540359903.log


In [2]:
#Load some house sales data
sales = graphlab.SFrame('home_data.gl/')

In [3]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900,3,1,1180,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000,3,2.25,2570,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000,2,1,770,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000,4,3,1960,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000,3,2,1680,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000,4,4.5,5420,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500,3,2.25,1715,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850,3,1.5,1060,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500,3,1,1780,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000,3,2.5,1890,6560,2,0


In [4]:
##Create Train and Test Dataset
train_data,test_data = sales.random_split(.8,seed=0)

In [24]:
from itertools import groupby

In [21]:
##1.Selection and Summary Statistics 
zipcode = sales['zipcode'].unique()
length=len(zipcode)
zipcode
#Try
sales[sales['zipcode'== zipcode[2]]]['price'].mean()

540088.1419053345

In [27]:
#Problem with Groupby Function
#zipcode_group = sales.groupby('zipcode')
#average_price_zipcode = zipcode_group['price'].mean
highest = sales[sales['zipcode']=='98039']
highest['price'].mean()

2160606.5999999996

In [49]:
inRange = sales[(sales['sqft_living']>=2000) & (sales['sqft_living']<=4000)]

inRange.num_rows() 

sales.num_rows() 

21613

In [64]:
##2. Filtering data
from fractions import Fraction 
total = sales.num_rows()
count = sales[(sales['sqft_living']>=2000) & (sales['sqft_living']<=4000)]
count = count.num_rows()
float(count)/float(total)

0.4266413732475825

In [65]:
##3.Building a regression model with several more features
#Train the data
sqft_model = graphlab.linear_regression.create(train_data, target='price', features=['sqft_living'],validation_set=None)


Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 1

Number of unpacked features : 1

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.014315     | 4349521.926170     | 262943.613754 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [66]:
print sqft_model.evaluate(test_data)

{'max_error': 4143550.8825285938, 'rmse': 255191.02870527358}


In [69]:
#More features
my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
my_features_model = graphlab.linear_regression.create(train_data,target='price',features=my_features,validation_set=None)



Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.058852     | 3763208.270523     | 181908.848367 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [70]:
my_features_model.evaluate(test_data)

{'max_error': 3486584.509381705, 'rmse': 179542.4333126903}

In [71]:
advanced_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode','condition', 'grade','waterfront','view','sqft_above','sqft_basement','yr_built','yr_renovated','lat','long','sqft_living15','sqft_lot15'] 

In [76]:
advance_features_model = graphlab.linear_regression.create(train_data,target='price',features=advanced_features,validation_set=None)



Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 18

Number of unpacked features : 18

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.136941     | 3469012.450686     | 154580.940736 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [77]:
advance_features_model.evaluate(test_data)

{'max_error': 3556849.413858208, 'rmse': 156831.1168021901}

In [79]:
#The advanced feature model has smaller rmse
print my_features_model.evaluate(test_data)['rmse']-advance_features_model.evaluate(test_data)['rmse']

22711.3165105
